# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

# Data Fetching

In [2]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16],low_memory=False)
    out= pd.read_csv('../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19],low_memory=False)
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


# A fast algorithm for Independent Component Analysis Decomposition

In [3]:
from sklearn.decomposition import FastICA
import warnings
scaler_obj1=FastICA()
scaler_obj2=FastICA()
X1=scaler_obj1.fit_transform(A1)
Y1=scaler_obj2.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

# Parameter Tuning

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [7,8,10,],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'n_estimators' : [80,90,100,200,400,1000],
        'objective': ['reg:squarederror','count:poisson']
    }

    xgb_model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)

    gsearch = RandomizedSearchCV(estimator = xgb_model, param_distributions = param_tuning, n_iter = 5, cv = 2, verbose=2, random_state=0, n_jobs = -1)

    
    grid_result = MultiOutputRegressor(gsearch).fit(x_train, y_train)

    return grid_result.estimators_[0].best_params_

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=0)

params=hyperParameterTuning(x_train,y_train)
params

# Model

In [4]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(XGBRegressor(tree_method='gpu_hist', gpu_id=0,max_depth=8,cosample_bytree=.5,learning_rate=.1,min_child_weight=3,
                   n_estimators=200,subsample=.7))

#training the model
model_fit1=model1.fit(x_train, y_train)
print("Model training is Done!!")

#saving as file
filename1 = 'xgboost.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

[14:07:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:07:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:08:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be use

# Error Analysis

In [5]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')

r2 score on train data 0.6324529971892136
r2 score on test data 0.14156921414635998
Mean Absolute Error: 0.00012905022753085157
Mean Squared Error: 1.915377379008494e-07
Root Mean Squared Error: 0.00043765024608795707
 



# Prediction of particles

In [6]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("xgboost.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[-5.7199410e-05  2.7625161e-05 -1.2138559e-06 ... -2.4311650e-05
  -1.1513624e-04  3.1531112e-05]
 [ 1.4481033e-04  5.3320367e-05 -5.2308377e-07 ... -5.7751557e-05
  -9.4005292e-05 -2.6711590e-05]
 [ 8.8566187e-05  5.0223760e-05 -9.1260392e-07 ... -2.7807588e-05
   5.3635649e-06 -3.0126119e-05]
 ...
 [-1.9085755e-06 -4.1146604e-06  6.4858995e-07 ...  1.9320785e-05
  -4.2327512e-05 -9.2818900e-06]
 [-6.1971041e-05  3.8131759e-05  8.3124638e-07 ... -2.1323616e-05
  -1.3361390e-04  2.2658489e-05]
 [ 1.8561255e-04  3.4789430e-06 -1.3847192e-06 ... -3.3684340e-05
   6.7150089e-05  1.2614171e-05]]


R2 Score :  0.14156921414635998


In [7]:
x_test=scaler_obj1.inverse_transform(x_test)
y_test_pred=scaler_obj2.inverse_transform(y_test_pred)
y_test=scaler_obj2.inverse_transform(y_test)
pd.DataFrame(y_test_pred)

,0,1,2,3,4,5,6
0,86.498520,632.667113,-73.147263,717.784748,4.273713,4.927039,5.721196
1,-128.010839,-2035.074324,11.818634,385.640413,15.764338,17.754541,20.287594
2,328.161695,573.881641,-103.818482,226.303236,22.111039,25.590532,28.656824
3,82.322598,1402.864927,-67.111745,672.756982,13.190700,15.621035,19.290451
4,40.507800,-2762.034270,-50.703266,174.752272,22.709509,26.048118,28.652838
...,...,...,...,...,...,...,...
432566,-216.710036,-2691.250336,81.847611,40.110101,6.013289,6.474002,8.020694
432567,130.376889,290.912553,-95.292659,585.990310,2.534069,2.903450,3.609272
432568,-144.284571,-4251.106144,-12.592735,374.096489,31.909968,37.270690,39.692084
432569,134.803318,58.695636,-97.944594,571.105188,1.938992,2.155027,2.721193


In [8]:
import datetime
from datetime import datetime
temp_arr=x_test[0:,1]
dates=list()
for i in range(0,len(temp_arr)):
    datetime_str=str(int(temp_arr[i]))
    datetime_str=datetime_str[0:6]
    if(datetime_str[4:6]=="00"):
        datetime_str=datetime_str[0:4]+'1'
    datetime_obj = datetime.strptime(datetime_str,"%y%m%d")
    dates.append(str(datetime_obj.date()))


In [9]:
import pandas as pd
df1=pd.DataFrame(y_test,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df1.index.name="DATE"
print("Actual Values:")
df1

Actual Values:


,NO2,O3,NO,CO,PM1,PM2.5,PM10
DATE,,,,,,,
2019-06-03,82.0,36.0,-79.0,706.000000,2.000000,2.420000,2.820000
2019-09-25,19.0,50.0,-8.0,478.000000,10.170000,10.950000,12.670000
2019-05-13,58.0,38.0,-34.0,-107.000031,27.400000,31.750000,41.180000
2019-04-18,83.0,5.0,-27.0,1579.999969,39.410000,45.180000,50.160000
2019-08-30,5.0,54.0,-4.0,-170.000031,6.900000,8.250000,9.110000
...,...,...,...,...,...,...,...
2019-03-28,-3.0,104.0,64.0,29.000000,7.130000,7.980000,8.190000
2019-06-29,77.0,40.0,-81.0,413.000000,1.940001,2.610001,4.120001
2019-10-02,21.0,3.0,-48.0,-3.000000,52.910002,61.519999,67.230001


In [10]:
arr=["Y_Actual"]*25
temp_df1=df1.head(25)
temp_df1['Data']=arr
temp_df1.to_excel("xgboost_y_test.xlsx")
temp_df1

C:\Users\utkar\AppData\Local\Temp/ipykernel_4388/2545994415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df1['Data']=arr


,NO2,O3,NO,CO,PM1,PM2.5,PM10,Data
DATE,,,,,,,,
2019-06-03,82.0,36.0,-79.0,706.000000,2.000000,2.420000,2.820000,Y_Actual
2019-09-25,19.0,50.0,-8.0,478.000000,10.170000,10.950000,12.670000,Y_Actual
2019-05-13,58.0,38.0,-34.0,-107.000031,27.400000,31.750000,41.180000,Y_Actual
2019-04-18,83.0,5.0,-27.0,1579.999969,39.410000,45.180000,50.160000,Y_Actual
2019-08-30,5.0,54.0,-4.0,-170.000031,6.900000,8.250000,9.110000,Y_Actual
2019-06-29,10.0,34.0,35.0,52.000000,6.889999,8.320000,10.110000,Y_Actual
2019-07-28,24.0,6.0,-38.0,-69.000000,18.180000,22.040001,23.080000,Y_Actual
2019-07-09,72.0,12.0,-93.0,649.000000,3.120000,4.330000,6.960001,Y_Actual
2019-09-06,52.0,66.0,-73.0,244.000000,20.639999,23.180000,23.709999,Y_Actual


In [11]:
import pandas as pd
df2=pd.DataFrame(y_test_pred,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df2.index.name="DATE"
print("Predicted Values:")
df2

Predicted Values:


,NO2,O3,NO,CO,PM1,PM2.5,PM10
DATE,,,,,,,
2019-06-03,86.498520,632.667113,-73.147263,717.784748,4.273713,4.927039,5.721196
2019-09-25,-128.010839,-2035.074324,11.818634,385.640413,15.764338,17.754541,20.287594
2019-05-13,328.161695,573.881641,-103.818482,226.303236,22.111039,25.590532,28.656824
2019-04-18,82.322598,1402.864927,-67.111745,672.756982,13.190700,15.621035,19.290451
2019-08-30,40.507800,-2762.034270,-50.703266,174.752272,22.709509,26.048118,28.652838
...,...,...,...,...,...,...,...
2019-03-28,-216.710036,-2691.250336,81.847611,40.110101,6.013289,6.474002,8.020694
2019-06-29,130.376889,290.912553,-95.292659,585.990310,2.534069,2.903450,3.609272
2019-10-02,-144.284571,-4251.106144,-12.592735,374.096489,31.909968,37.270690,39.692084


In [12]:
arr=["Y_Predicted"]*25
temp_df2=df2.head(25)
temp_df2['Data']=arr
temp_df2.to_excel("xgboost_y_test_pred.xlsx")
temp_df2

C:\Users\utkar\AppData\Local\Temp/ipykernel_4388/3694748081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df2['Data']=arr


,NO2,O3,NO,CO,PM1,PM2.5,PM10,Data
DATE,,,,,,,,
2019-06-03,86.498520,632.667113,-73.147263,717.784748,4.273713,4.927039,5.721196,Y_Predicted
2019-09-25,-128.010839,-2035.074324,11.818634,385.640413,15.764338,17.754541,20.287594,Y_Predicted
2019-05-13,328.161695,573.881641,-103.818482,226.303236,22.111039,25.590532,28.656824,Y_Predicted
2019-04-18,82.322598,1402.864927,-67.111745,672.756982,13.190700,15.621035,19.290451,Y_Predicted
2019-08-30,40.507800,-2762.034270,-50.703266,174.752272,22.709509,26.048118,28.652838,Y_Predicted
2019-06-29,1.133919,-1158.751586,5.253900,97.012636,7.041149,7.973965,10.246741,Y_Predicted
2019-07-28,-7.695644,-1084.773397,-20.686683,259.247259,25.357452,29.688782,31.634597,Y_Predicted
2019-07-09,77.614479,208.289623,-84.130059,695.450443,4.464519,5.152682,5.809915,Y_Predicted
2019-09-06,-41.149735,-2698.158378,-62.809816,502.248203,23.290084,27.191585,30.792031,Y_Predicted


In [13]:
#Completed